In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
DEBUG=False

In [ ]:
%run ./nb_core/root_dirs.ipynb
setup_syspath_disentangle(DEBUG)
%run ./nb_core/disentangle_imports.ipynb

In [ ]:
nmodel_dir = '/home/ashesh.ashesh/training/noise_model/2402/350/'
# nmodel_dir = '/home/ashesh.ashesh/training/noise_model/2402/61'

histnoisemodel_fpath = None
gmmnoisemodel_fpath = None
for fname in os.listdir(nmodel_dir):
    if fname.startswith('HistNoiseModel'):
        assert histnoisemodel_fpath is None
        histnoisemodel_fpath = os.path.join(nmodel_dir, fname)
    elif fname.startswith('GMMNoiseModel'):
        assert gmmnoisemodel_fpath is None
        gmmnoisemodel_fpath = os.path.join(nmodel_dir, fname)
print(gmmnoisemodel_fpath)
print(histnoisemodel_fpath)

In [ ]:
from disentangle.nets.gmm_noise_model import GaussianMixtureNoiseModel
from disentangle.nets.hist_noise_model import HistNoiseModel

# gmmnoisemodel_fpath = '/home/ashesh.ashesh/training/noise_model/2402/62/GMMNoiseModel_CCPs-GT_all.mrc__6_4_Clip0.0-1.0_Sig0.125_UpNone_Norm0_bootstrap.npz'
# histnoisemodel_fpath = os.path.join(os.path.dirname(gmmnoisemodel_fpath), 'HistNoiseModel_CCPs-GT_all.mrc__Norm0_Bins128_bootstrap.npy')
# datadir = '/group/jug/ashesh/data/ventura_gigascience/actin-60x-noise2-highsnr.tif' if 'actin' in os.path.basename(gmmnoisemodel_fpath) else '/group/jug/ashesh/data/ventura_gigascience/mito-60x-noise2-highsnr.tif'


In [ ]:
nmodel_params = np.load(gmmnoisemodel_fpath)
gmm_model = GaussianMixtureNoiseModel(params=nmodel_params)
histdata = np.load(histnoisemodel_fpath)
hist_model = HistNoiseModel(histdata)
bins = histdata.shape[-1]

In [ ]:
from disentangle.utils import plotProbabilityDistribution
signalBinIndex=40
data_dict = plotProbabilityDistribution(signalBinIndex=signalBinIndex, 
                            histogramNoiseModel=hist_model,
                            gaussianMixtureNoiseModel=gmm_model,
                            device='cpu')
data_dict['gmm']['x'][data_dict['gmm']['p'].argmax()]

In [ ]:
# for i in range(histdata.shape[1]):
#     assert np.std(histdata[1][i]) < 1e-7
#     assert np.std(histdata[2][i]) < 1e-7

In [ ]:
# bin_val = (histdata[1] + histdata[2])/2
# bin_val = bin_val[:,0]
# binsize = np.mean(histdata[2] - histdata[1])
# bin_pdf = histdata[0]/binsize

In [ ]:
# from scipy.optimize import curve_fit
# import math
# import numpy as np

# def gaus(x, mu,sigma):
#     out = np.exp(-(x-mu)**2/(2*sigma**2)) * 1/(sigma*np.sqrt(2*math.pi))
#     # print(out.shape, out.min(), out.max())
#     return out

# def sigmoid(x):
#   return 1 / (1 + math.exp(-x))

In [ ]:
# index = 90
# x = bin_val
# y = bin_pdf[index]

# mean =bin_val[index]
# sigma = sum(y*(x-mean)**2)/len(y)

# popt,pcov = curve_fit(gaus,
#                       x,
#                       y,
#                       p0=[x[index],sigma])


In [ ]:
# pcov

In [ ]:
# popt

In [ ]:
# plt.plot(bin_val,bin_pdf[index],'b+:',label='data')
# plt.plot(bin_val,gaus(bin_val,*popt),'ro:',label='fit')
# plt.legend()
# plt.title('Fig. 3 - Fit for Time Constant')
# plt.xlabel('Time (s)')
# plt.ylabel('Voltage (V)')
# plt.show()

In [ ]:

from scipy.optimize import curve_fit
import math
import numpy as np
import torch

def gaus(x, mu,sigma):
    out = np.exp(-(x-mu)**2/(2*sigma**2)) * 1/(sigma*np.sqrt(2*math.pi))
    return out

def gaus_pytorch(x, mu,sigma):
    print(mu, sigma)
    out = torch.exp(-(x-mu)**2/(2*sigma**2)) * 1/(sigma*np.sqrt(2*math.pi))
    return out

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

class GaussianNoiseModelV2:
    def __init__(self, histdata) -> None:
        self._histdata = histdata
        bin_val = (self._histdata[1] + self._histdata[2])/2
        # midpoint of every bin
        self._bin_val = bin_val[:,0]
        self._binsize = np.mean(self._histdata[2] - self._histdata[1])
        # probability density function.
        self._bin_pdf = self._histdata[0]/self._binsize
        self._params = []

        self.minv = np.min(histdata[1, ...])

        # The upper boundaries of each bin in y are stored in dimension 2
        self.maxv = np.max(histdata[2, ...])
        self.bins = histdata.shape[1]
        self._min_valid_index = None
        self._max_valid_index = None

    def fit_index(self, index):
        x = self._bin_val
        y = self._bin_pdf[index]
        if y.sum() * self._binsize < 1e-5:
            return torch.tensor([torch.nan, torch.nan])
        
        if self._min_valid_index is not None:
            self._min_valid_index = min(index, self._min_valid_index)
        else:
            self._min_valid_index = index
        
        if self._max_valid_index is not None:
            self._max_valid_index = max(index, self._max_valid_index)
        else:
            self._max_valid_index = index
        
        assert abs(y.sum() * self._binsize -1) < 1e-5

        mean =self._bin_val[index]
        sigma = sum(y*(x-mean)**2)/len(y)
        popt,pcov = curve_fit(gaus,
                            x,
                            y,
                            p0=[x[index],sigma])
        return torch.Tensor(popt)
    
    def fit(self):
        for index in range(len(self._bin_pdf)):
            popt = self.fit_index(index)
            self._params.append(popt)
        
        self._params = torch.stack(self._params)
        # manually adde after last and before first bin.
        if self._min_valid_index > 0:
            self._params[self._min_valid_index -1 ] = self._params[self._min_valid_index]
            self._params[self._min_valid_index -1, 0] -= self._binsize
            self._min_valid_index -= 1
        
        if self._max_valid_index < self.bins - 1:
            self._params[self._max_valid_index + 1] = self._params[self._max_valid_index]
            self._params[self._max_valid_index + 1, 0] += self._binsize
            self._max_valid_index += 1

    def getIndexSignalFloat(self, x):
        return torch.clamp(self.bins * (x - self.minv) / (self.maxv - self.minv), min=0.0, max=self.bins - 1 - 1e-3)

    def likelihood(self, obs, signal):
        signalF = self.getIndexSignalFloat(signal)
        signal_ = signalF.floor().long()
        fact = signalF - signal_.float()
        underflow_mask = signal_ < self._min_valid_index
        signal_[underflow_mask] = self._min_valid_index
        overflow_mask = signal_  > self._max_valid_index
        signal_[overflow_mask] = self._max_valid_index
        mu1, sigma1 = self._params[signal_]
        
        # if the signal is in the last bin, we just need to ignore the first mu and sigma and go with the last one.
        last_index_mask = signal_ == self._max_valid_index        
        signal_[last_index_mask] = self._max_valid_index - 1
        fact[last_index_mask] = 1.0

        mu2, sigma2 = self._params[signal_ + 1]
        mu = mu1 * (1 - fact) + mu2 * fact
        sigma = sigma1 * (1 - fact) + sigma2 * fact
        return gaus_pytorch(obs, mu, sigma)


In [ ]:
nm = GaussianNoiseModelV2(histdata)
nm.fit()

In [ ]:

max_signal = hist_model.maxv.item()
min_signal = hist_model.minv.item()
n_bin = int(hist_model.bins.item())

histBinSize = (max_signal - min_signal) / n_bin
querySignal_numpy = (signalBinIndex / float(n_bin) * (max_signal - min_signal) + min_signal)


In [ ]:
nm._min_valid_index

In [ ]:
signalBinIndex = 112
data_dict = plotProbabilityDistribution(signalBinIndex=signalBinIndex, 
                            histogramNoiseModel=hist_model,
                            gaussianMixtureNoiseModel=nm,
                            device='cpu')
data_dict['gmm']['x'][data_dict['gmm']['p'].argmax()]

In [ ]:
nm._min_valid_index

In [ ]:
nm._params[42]

In [ ]:
nm._binsize